# Checking simulation parameters, and the quality of simulation
- Energy and pressure plots
- Distance between periodic images


In [ ]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl
import panedr
import matplotlib.pyplot as plt

#Reloading funcs if changed
from importlib import reload 
import funcs,config
reload(funcs)
reload(config)

import xvg_plot as xvg
import xpm_plot as xpm



%matplotlib inline
from IPython.display import Markdown, display
from config import folder,descr,pname,ssh_host,set_gmx_cmd
display(Markdown(descr))
print("Folder set to: ",folder)
print("Project name: ",pname)



Quick view

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/7_prod.xtc')

# QC analysis
## Let's look at energy, temperature pressure

In [ ]:
df = panedr.edr_to_df('GMX_run/7_prod.edr', verbose=True)
num_of_keys=len(df.keys())
grid=[1+num_of_keys//3,3]

plt.figure(figsize=(grid[1]*6,grid[0]*3))
plt.title('Mod')
for i,key in enumerate(df.keys(),1):
    ax=plt.subplot(*grid, i)
    ax.plot(df[key][0:1000],label=key)
    ax.set_xlabel('Time, ps')
    ax.legend()    
plt.show()

#### All trajectory, every 1000 step

In [ ]:
step=1000 #plot every 1000 records

plt.figure(figsize=(grid[1]*6,grid[0]*3))
plt.title('Mod')
for i,key in enumerate(df.keys(),1):
    ax=plt.subplot(*grid, i)
    ax.plot(df[key][0::step],label=key)
    ax.set_xlabel('Time, ps')
    ax.legend()    
plt.show()

### Checking periodic images
- We need to check if there are problems in periodic image interactions
- http://manual.gromacs.org/documentation/2018/onlinehelp/gmx-mindist.html

Our initial box vectors of a truncated octahedron are
gro-file notation
10.23860   9.65312   8.36010   0.00000   0.00000   3.41266   0.00000  -3.41266   4.82612
 ax          by        cz        ay         az         bx       by        cx        cy
     a    b   c  
 x   ax   bx  cx  | d    d/3            -d/3      
 y   ay   by  cy  | 0    d*2*sqrt(2)/3  d*sqrt(2)/3
 z   az   bz  cz  | 0    0              d*sqrt(6)/3 

In [ ]:
%%bash  -s "$set_gmx_cmd"
$1
mkdir -p big_data
#gmx mindist -n GMX_system/index.ndx -f GMX_run/7_prod.xtc -s GMX_run/7_prod.tpr -od big_data/mindist.xvg -pi -dt 10000 << !
#gmx mindist -n GMX_system/index.ndx -f GMX_run/7_prod.xtc -s GMX_run/7_prod.tpr -od big_data/mindist.xvg -pi -dt 10000 << !
gmx mindist -n GMX_system/index.ndx -f tmp/7_prod_2.xtc -s GMX_run/7_prod.tpr -od big_data/mindist.xvg -pi -dt 10000 << !
!Water_and_ions
!

In [ ]:
#With option -pi the minimum distance of a group to its periodic image is plotted. 
#This is useful for checking if a protein has seen its periodic image during a simulation.
#Only one shift in each direction is considered, giving a total of 26 shifts.
#It also plots the maximum distance within the group and the lengths of the three box vectors.

metadata, data = xvg.parse_xvg('big_data/mindist.xvg', 'all')
data=np.array(data)
#data=data[:,0:10]
xvg.plot_data(data, metadata,figsize=(15,15))